In [ ]:
#Import libraries

#basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
plt.style.use('ggplot')

#nlp
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
# !pip3 install emoji
import emoji
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download('punkt')
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
# nltk.download('averaged_perceptron_tagger')
from nltk.corpus import treebank_chunk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

#ml
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import optuna

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

In [28]:
df = pd.read_csv('airlines_reviews.csv')
display(df.head(1), df.shape)


,Title,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended
0,Flight was amazing,Alison Soetantyo,2024-03-01,Singapore Airlines,True,Flight was amazing. The crew onboard this fl...,Solo Leisure,December 2023,Jakarta to Singapore,Business Class,4,4,4,4,4,9,yes


(8100, 17)

### 1. Preprocessing function

In [ ]:

# Define the preprocessing function
def preprocess_reviews(text):
    # Convert text to lowercase
    text = text.astype(str).str.lower()
    
    # Remove emojis
    text = text.apply(lambda x: emoji.demojize(x))
    text = text.str.replace(r':[a-z_]+:', ' ', regex=True)
    
    # Remove special characters and numbers
    text = text.str.replace(r'[^a-zA-Z\s]', '', regex=True)
    
    # Tokenization (split the text into sentences)
    sentences = text.apply(lambda x: sent_tokenize(x))
    
    # Flatten list of sentences
    sentences = sentences.explode()
    
    # Tokenize sentences into words
    words = sentences.apply(lambda x: word_tokenize(x))
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = words.apply(lambda x: [word for word in x if word not in stop_words])
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_words = words.apply(lambda x: [stemmer.stem(word) for word in x])
    
    # Join the words back into a single string
    preprocessed_text = stemmed_words.apply(lambda x: ' '.join(x))
    
    return preprocessed_text

### 2. Random forest hyperparamter optimisation with peanlty on Neutral class misclassification

In [ ]:
# defining train and test sets
X = df['Reviews']
y = df['Overall Rating'].apply(lambda x: 0 if x <= 4 else 1 if x <= 6 else 2)

print(y.value_counts())

# Preprocess text data
X_preprocessed = preprocess_reviews(X)

# import preprocessing
# X_preprocessed = preprocessing.preprocess_reviews(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Vectorize preprocessed text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


# Hyperparameter optimization using Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    rf_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        class_weight={0: 1, 1: 5, 2: 1},
        random_state=42
    )
    
    val_scores = cross_val_score(rf_model, X_train_tfidf, y_train, cv=10, scoring='f1_weighted')
    return val_scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=25)

# Train Random Forest model with best parameters
best_params = study.best_params
rf_model = RandomForestClassifier(**best_params, class_weight={0: 1, 1: 5, 2: 1}, random_state=42)
rf_model.fit(X_train_tfidf, y_train)

# Perform 10-fold cross-validation
val_scores = cross_val_score(rf_model, X_train_tfidf, y_train, cv=10, scoring='f1_weighted')

# Print the mean accuracy and standard deviation
print("Mean val f1 score:", val_scores.mean())
print("Val Standard Deviation:", val_scores.std())

# Predict ratings
y_pred = rf_model.predict(X_test_tfidf)

# Print classification report
print(classification_report(y_test, y_pred))

AttributeError: module 'preprocessing' has no attribute 'preprocess_reviews'

### 3.0 Logistic Regression 

In [ ]:
# defining train and test sets
X = df['Reviews']
y = df['Overall Rating'].apply(lambda x: 0 if x <= 4 else 1 if x <= 6 else 2)

print(y.value_counts())

# Preprocess text data
X_preprocessed = preprocess_reviews(X)

# import preprocessing
# X_preprocessed = preprocessing.preprocess_reviews(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Vectorize preprocessed text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Define objective function for Optuna
def objective(trial):
    # Define hyperparameters to optimize
    penalty = trial.suggest_categorical('penalty', ['l2'])
    C = trial.suggest_loguniform('C', 0.001, 10)  
    max_iter = trial.suggest_int('max_iter', 100, 1000)  
    
    # Define class weights with a higher penalty on class 1
    class_weight = {0: 1, 1: 10, 2: 0.5}
    
    # Train Logistic Regression model with hyperparameters
    lr_model = LogisticRegression(multi_class='ovr', solver='lbfgs', penalty=penalty, C=C, max_iter=max_iter,
                                  class_weight=class_weight, random_state=42)
    val_scores = cross_val_score(lr_model, X_train_tfidf, y_train, cv=5, scoring='f1_weighted')
    
    # Return mean validation accuracy as the objective value to maximize
    return val_scores.mean()

# Optimize hyperparameters using Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best hyperparameters
best_params = study.best_params
print(best_params)

# Train Logistic Regression model with best hyperparameters
class_weight = {0: 1, 1: 10, 2: 0.5}  # Penalty on class 1
lr_model = LogisticRegression(multi_class='ovr', solver='lbfgs', class_weight=class_weight, **best_params, random_state=42)
lr_model.fit(X_train_tfidf, y_train)

# Predict ratings
y_pred = lr_model.predict(X_test_tfidf)

# Print classification report
print(classification_report(y_test, y_pred))

Overall Rating
2    3915
0    3403
1     782
Name: count, dtype: int64


ValueError: not enough values to unpack (expected 2, got 1)

#### 3.1 Hyperparameter Optimisation Logistic Regression 

In [181]:
# import optuna
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import accuracy_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split

# # defining train and test sets
# X = df['Reviews']
# y = df['Overall Rating'].apply(lambda x: 1 if x >= 6.9 and x <= 11 else
#                                     0 if x >= 4.0 and x <= 6.1 else
#                                     -1 )
# print(y.value_counts())

# # Preprocess text data
# X_preprocessed = sentence_level_preprocessing(X)

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# # Define objective function for Optuna
# def objective(trial):
#     # Define hyperparameters to optimize
#     penalty = trial.suggest_categorical('penalty', ['l2'])  # Regularization type
#     C = trial.suggest_loguniform('C', 0.001, 10)  # Regularization strength
#     max_iter = trial.suggest_int('max_iter', 100, 1000)  # Maximum number of iterations
#     class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])  # Class weights
    
#     # Train Logistic Regression model with hyperparameters
#     lr_model = LogisticRegression(multi_class='ovr', solver='lbfgs', penalty=penalty, C=C, max_iter=max_iter,
#                                   class_weight=class_weight, random_state=42)
#     val_scores = cross_val_score(lr_model, X_train_tfidf, y_train, cv=5, scoring='accuracy')
    
#     # Return mean validation accuracy as the objective value to maximize
#     return val_scores.mean()

# # Optimize hyperparameters using Optuna
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get the best hyperparameters
# best_params = study.best_params
# print(best_params)

# # Train Logistic Regression model with best hyperparameters
# lr_model = LogisticRegression(multi_class='ovr', solver='lbfgs', **best_params, random_state=42)
# lr_model.fit(X_train_tfidf, y_train)

# # Predict ratings
# y_pred = lr_model.predict(X_test_tfidf)

# # Calculate accuracy
# acc = accuracy_score(y_test, y_pred)
# print('Prediction accuracy:', acc)

# #best acc 80.37

Overall Rating
 1    3915
-1    3022
 0    1163
Name: count, dtype: int64


[I 2024-04-28 17:58:35,147] A new study created in memory with name: no-name-05f891c6-9a0a-4730-8e3c-1538d9cbf765
/var/folders/yj/2fr85zvn2sbbpv3jsq4y3kwh0000gn/T/ipykernel_5316/1774006226.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.001, 10)  # Regularization strength
[I 2024-04-28 17:58:36,014] Trial 0 finished with value: 0.7725308641975308 and parameters: {'penalty': 'l2', 'C': 5.3225364310590635, 'max_iter': 103, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7725308641975308.
/var/folders/yj/2fr85zvn2sbbpv3jsq4y3kwh0000gn/T/ipykernel_5316/1774006226.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = tri

Prediction accuracy: 0.8037037037037037
